In [1]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
%matplotlib qt

class bird:
    def updateVelocity(self):
        self.velocity = np.sqrt(self.vx**2 + self.vy**2)

    def __init__(self, radius):
        ''' radius is the radius of the circle to use for the boundary conditions'''

        #specify the max radius
        self.maxR = radius
        
        #Specify a parameter for square of maximum inital velocity
        self.maxVelocity =  30


        #generate a random (x,y) coordinate and (vx, vy) coordinate for the inital condition of bird
        #scale is how much smaller the size of the area the birds start in is than the whole arena. scale = .5 -> r_start = 1/2*maxR
        scale = 0.4

        #generate a random radius within the size allotted to be the initial radius of this bird
        r = (np.random.rand())*self.maxR*scale
        theta = (np.random.rand())*2*np.pi
        self.x = r*np.cos(theta)
        self.y = r*np.sin(theta)

        #allow the velocity to be positive or negative, [vxScale, vyScale in (-1,1)]
        vxScale = (2*np.random.rand() - 1)
        vyScale = (2*np.random.rand() - 1)
        self.vx = vxScale*self.maxVelocity
        self.vy = vyScale*self.maxVelocity 
        
        self.updateVelocity()
    
    def updatePos(self, birdList):
        global numBirds
        dt = 0.0001

        c = 1

        #drag coefficient
        d = 0.000001

        r = .001

        
        #specify an angle for the width of field of view
        fovTheta = np.pi/5

        birdsInView = []
        avgXinView = 0
        avgYinView = 0


        for i in range(numBirds):
            #compute the dot product of the birds velocity with the position of other bird relative to this bird

            #s is the vector from this bird (self) to the bird in the bird list
            sx = birdList[i].x - self.x
            sy = birdList[i].y - self.y

            #normalize S
            distanceToBird = np.sqrt(sx**2 + sy**2)
            if distanceToBird != 0:
                sx = sx/ distanceToBird
                sy = sy/ distanceToBird

                avgXinView -= (sx)/(distanceToBird**2)*r
                avgYinView -= (sy)/(distanceToBird**2)*r

            #normalize the velocity
            if self.velocity != 0:
                vxNorm = self.vx/self.velocity
                vyNorm = self.vy/self.velocity
            else:
                vxNorm = 0
                vyNorm = 0

            sDotv = sx*vxNorm + sy*vyNorm

            theta = np.arccos(sDotv)

            if theta < fovTheta/2:
                birdsInView.append(birdList[i])

                avgXinView += (sx)/distanceToBird*np.cos(theta) 
                avgYinView += (sy)/distanceToBird*np.cos(theta)

        numBirdsinView = len(birdsInView)
        '''if numBirdsinView > 0:
            avgXinView = avgXinView / numBirdsinView
            avgYinView = avgYinView / numBirdsinView'''

        self.vy = self.vy + c*avgYinView - d * self.vy
        self.vx = self.vx + c*avgXinView - d * self.vx

        self.updateVelocity()
        
        if self.velocity > self.maxVelocity:
            self.vx = self.vx * self.maxVelocity/self.velocity
            self.vy = self.vy * self.maxVelocity/self.velocity
            self.updateVelocity()
        
        self.x = self.x + self.vx * dt
        self.y = self.y + self.vy * dt

        radius = np.sqrt(self.x**2 + self.y**2)
        if (radius > self.maxR):
            self.x = self.x * self.maxR/radius
            self.y = self.y * self.maxR/radius

            nx = self.x/self.maxR
            ny = self.y/self.maxR
            vDotn = self.vx * nx + self.vy * ny
            self.vx = self.vx - 2*vDotn*nx
            self.vy = self.vy - 2*vDotn*ny



        

#specify dimensions of bird box     
rMax = 1

#initialize subplots      
fig, ax = plt.subplots()




ax.set_xlim((-rMax, rMax))
ax.set_ylim((-rMax, rMax))
ircle = plt.Circle((0,0),rMax, fill = False)
ax.add_patch(ircle)
#specify the number of birds
numBirds = 33

#create a list to hold the birds
birdList = np.zeros(numBirds).astype(bird)

#x and y data for plotting the birds
birdsXdata = np.zeros(numBirds)
birdsYdata = np.zeros(numBirds)

#create and plot the birds
for i in range(numBirds):
    birdList[i] = bird(rMax)
    birdsXdata[i] = birdList[i].x
    birdsYdata[i] = birdList[i].y

plottedBirds, =ax.plot(birdsXdata,birdsYdata, "ro")




def updateBirds():
    global t
    birdListCopy = birdList.copy()
    for i in range(numBirds):
        birdList[i].updatePos(birdListCopy)
        birdsXdata[i] = birdList[i].x
        birdsYdata[i] = birdList[i].y
       
    plottedBirds.set_data(birdsXdata, birdsYdata)

t = 0
def update(frame):
    global t
    updateBirds()
    t += .1
    return 0

anim = FuncAnimation(fig, update, interval =1)
ax.set_aspect('equal')


plt.show()

/var/folders/y6/gtpqx90n73l8kn79d102f4g00000gn/T/ipykernel_20577/189297562.py:176: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  anim = FuncAnimation(fig, update, interval =1)
